In [22]:
import os
import pandas as pd
import numpy as np
import anndata
from scipy.stats import zscore
import seaborn as sns
import matplotlib.pyplot as plt
plt.rcParams["font.family"] = "Arial"
import matplotlib
matplotlib.rcParams['pdf.fonttype'] = 42
import scanpy as sc

import re

from pyInfinityFlow.InfinityFlow_Utilities import marker_finder
from pyInfinityFlow.InfinityFlow_Utilities import pearson_corr_df_to_df


os.chdir("/media/kyle_storage/kyle_ferchen/grimes_lab_main/analysis/"\
    "2023_06_12_tea_seq_atac_processing/")

In [15]:
new_cluster_order = pd.read_csv("/media/kyle_storage/kyle_ferchen/"\
    "grimes_lab_main/data/2021_11_mouse_optimized_cite_seq/processed_files/"\
    "r7_cluster_order_kf_2024_01.csv")
map_r7_to_lv3 = pd.Series(\
    new_cluster_order["Level 3"].values,
    index=new_cluster_order["Cluster"].values)
map_lvl3_to_order = pd.Series(\
    new_cluster_order["Order"].values,
    index=new_cluster_order["Level 3"].values)
map_r7_to_replace_dash = pd.Series(\
    new_cluster_order["Cluster"].values,
    index=[i.replace("-", "_") for i in new_cluster_order["Cluster"].values])
new_cluster_order

,Cluster,Group,Old_order,CITE-to-TEA,Order,Level 1,Level 2,Level 3,Level Kairavee
0,LT-HSC_Mllt3,HSCP,1,LT-HSC_Mllt3,1.0,HSPC,HSC,qHSC,HSC
1,ST-HSC,HSCP,2,ST-HSC,2.0,HSPC,HSC,aHSC,HSC
2,MPP4-Hlf,HSCP,3,MPP4-Hlf,3.0,HSPC,MPP4,HSC-Ly,HSPC
3,MPP5-Egr1,HSCP,8,MPP5-Egr1,4.0,HSPC,MPP5,MPP5-IER,MPP5-IER
4,MPP5-Flt3,HSCP,7,MPP5-Flt3,5.0,HSPC,MPP5,MPP5 Ly-I,HSPC
...,...,...,...,...,...,...,...,...,...
83,ILC2,ILC,84,ILC2,84.0,ILC,ILC,ILC2,ILC
84,Bcl11b+_preETP_Cd3d,ILC,85,Bcl11b+_preETP_Cd3d,85.0,T cell,preETP,pre-ILC1-ILC3-NKP,preETP
85,Bcl11b+_preETP_Tdrd5,ILC,88,Bcl11b+_preETP_Tdrd5,86.0,T cell,preETP,pre-NKP,preETP
86,ILC1-ILC3-NKP,ILC,87,ILC1-ILC3-NKP,87.0,ILC,ILC,ILC1-ILC3-NKP,ILC


In [3]:
# Load CITE-seq data (SoupX 0.15 corrected)
path_to_cite_data = "/media/kyle_storage/kyle_ferchen/grimes_lab_main/data/"\
    "2021_11_mouse_optimized_cite_seq/processed_files/"

cite_adata = anndata.read_h5ad(os.path.join(\
    path_to_cite_data,
    "cite_seq_adata_rna_combined_SoupX_0_15_with_R7_clusters.h5ad"))

cite_adata.X = np.log2((10000 * (cite_adata.X.T / \
    cite_adata.X.sum(axis=1).T).T) + 1)

cite_adata.obs["lvl3"] = cite_adata.obs["R7"].replace(\
    map_r7_to_lv3.to_dict()).values
cite_adata.obs["cluster_order"] = cite_adata.obs["lvl3"].replace(\
    map_lvl3_to_order.to_dict()).values

cite_adata = cite_adata[\
    cite_adata.obs.loc[cite_adata.obs["lvl3"] != "MEP-UNK"].sort_values(\
        by="cluster_order").index.values,
    :]

cite_df = pd.DataFrame(\
    cite_adata.X,
    index=cite_adata.obs.index.values,
    columns=cite_adata.var.index.values)

cite_centroids = cite_df.copy()
cite_centroids["cluster"] = cite_adata.obs.loc[\
    cite_centroids.index.values, "lvl3"].values
cite_centroids = pd.pivot_table(cite_centroids, index="cluster", aggfunc=np.mean)

cite_adata

View of AnnData object with n_obs × n_vars = 55285 × 28692
    obs: 'barcode', 'port', 'R7', 'lvl3', 'cluster_order'
    var: 'gene', 'feature_type'

In [30]:
# Seqlet annotation
seqlets = pd.read_feather("output/chrombpnet/modisco_merged_results/fold_0/"\
    "redo_extract_seqlets/all_seqlit_hits_above_modisco_min_anno.fea")

# Seqlets
dp_scores = pd.read_feather("output/chrombpnet/modisco_merged_results/fold_0/"\
    "redo_extract_seqlets/all_seqlit_hits_above_modisco_min_dp_scores.fea")
dp_scores = dp_scores.set_index("index").astype(np.float32)
dp_scores = dp_scores.rename(map_r7_to_replace_dash.to_dict(), axis=1)
dp_scores = dp_scores.rename(map_r7_to_lv3.to_dict(), axis=1)

In [5]:
# Read in the TADs
tads = pd.read_table("input/GSE119347_BMHSC_TADs/GSE119347_BMHSC_TADs_mm10_liftover.bed", 
    header=None)
tads.columns = ["chr", "start", "end", "name", "score"]
tads

,chr,start,end,name,score
0,chr10,5881540,7341540,chr10:3005001-4465000,1
1,chr10,5321540,5881540,chr10:4465001-5025000,1
2,chr10,4401540,5321540,chr10:5025001-5945000,1
3,chr10,3961540,4401540,chr10:5945001-6385000,1
4,chr10,3100000,3961540,chr10:6385001-7265000,1
...,...,...,...,...,...
2511,chrX,162907068,164497068,chrX:159345001-160935000,1
2512,chrX,164497068,166137068,chrX:160935001-162575000,1
2513,chrX,166137068,167267068,chrX:162575001-163705000,1
2514,chrX,167267068,168537068,chrX:163705001-164975000,1


In [6]:
# Read in the ATAC data
path_atac_data = "output/tea_r7_pseudobulk_from_pmat/"

peaks = pd.read_table(\
    os.path.join(path_atac_data, "r7_tea_cluster_peak_set_with_tss.bed"),
    header=None)
peaks.columns = ["chr", "start", "end", "name", "score", "strand"]

atac_counts = pd.read_csv(\
    os.path.join(path_atac_data, "r7_tea_pbulk_cpm_from_binary_pmat.csv"))

path_split_bams = "output/tea_r7_split_bams/"

# Get the tea-seq metrics
cell_counts = {}
read_counts = {}
for item in os.listdir(path_split_bams):
    tmp_metrics = pd.read_table(os.path.join(path_split_bams, item,
         "metrics_bam_cluster_splits.txt"))
    for i, row in tmp_metrics.iterrows():
        if row["cluster"] in cell_counts:
            cell_counts[row["cluster"]] += row["cells_per_cluster"]
            read_counts[row["cluster"]] += row["reads_per_cluster"]
        else:
            cell_counts[row["cluster"]] = row["cells_per_cluster"]
            read_counts[row["cluster"]] = row["reads_per_cluster"]

tea_metrics = pd.concat([\
    pd.Series(cell_counts), 
    pd.Series(read_counts)], axis=1)
tea_metrics.columns = ["cells", "reads"]
tea_metrics.head()

# Only select clusters with at least 100 cells
sel_clusters = tea_metrics.loc[tea_metrics["cells"] >= 100].index.values
atac_counts = atac_counts[sel_clusters]

# Get max log2 cpm observed for each peak
peaks_to_index = pd.DataFrame({\
    "i": peaks.index.values,
    "peak": (peaks["chr"] + ":" + peaks["start"].astype(str) + "-" + \
        peaks["end"].astype(str)).values,
    "max_cpm": atac_counts.max(axis=1),
    "top_cluster": atac_counts.idxmax(axis=1)})

# Only accept peaks with a log2-cpm value > 3
expressed_peaks = peaks_to_index.loc[\
    peaks_to_index["max_cpm"] > 3]["peak"].values

In [7]:
# Pull out the TSS positions from peaks
tss_loci = peaks.loc[peaks["name"].str.contains("TSS:")].copy()
tss_loci["gene"] = [i.split(":")[-1] for i in tss_loci["name"].values]
tss_loci = tss_loci[["gene", "chr", "start"]]
tss_loci["start"] = tss_loci["start"] + 500
tss_loci

,gene,chr,start
114,Xkr4,chr1,3671497
373,Mrpl15,chr1,4785709
382,Lypla1,chr1,4807822
404,Tcea1,chr1,4857813
442,Rgs20,chr1,5018734
...,...,...,...
751317,Uty,chrY,1245690
751334,Ddx3y,chrY,1286628
751345,Usp9y,chrY,1459781
757491,Gm21860,chrY,90755466


In [8]:
## Assign each atac peak a TAD
print("Assigning TADs to ATAC peaks...")
atac_tad_assignments = pd.Series([np.nan] * peaks.shape[0], 
    index=peaks.index.values)
for tmp_chr in tads["chr"].unique():
    print("\tWorking on {}...".format(tmp_chr))
    seg_tads = tads.loc[tads["chr"] == tmp_chr].copy()
    seg_peaks = peaks.loc[peaks["chr"] == tmp_chr].copy()
    for i, tmp_tad in seg_tads.iterrows():
        tmp_hits = ~((tmp_tad["start"] > seg_peaks["end"]) | \
            (tmp_tad["end"] < seg_peaks["start"]))
        tmp_hits = tmp_hits[tmp_hits].index.values
        if len(tmp_hits) > 0:
            atac_tad_assignments.loc[tmp_hits] = i

atac_tad_assignments = atac_tad_assignments[\
    ~atac_tad_assignments.isna()].astype(int)

peaks["tad"] = ""
peaks.loc[atac_tad_assignments.index.values, "tad"] = \
    (tads.loc[atac_tad_assignments.values, "chr"] + ":" + \
    tads.loc[atac_tad_assignments.values, "start"].astype(str) + "-" + \
    tads.loc[atac_tad_assignments.values, "end"].astype(str)).values

Assigning TADs to ATAC peaks...
	Working on chr10...
	Working on chr11...
	Working on chr12...
	Working on chr19...
	Working on chrX...
	Working on chr8...
	Working on chr13...
	Working on chr5...
	Working on chr2...
	Working on chr6...
	Working on chr14...
	Working on chr15...
	Working on chr18...
	Working on chr16...
	Working on chr17...
	Working on chr4...
	Working on chr1...
	Working on chr3...
	Working on chr7...
	Working on chr9...
	Working on chrX_GL456233_random...
	Working on chrY...


In [10]:
# Assign TADs to tss start sites
print("Assigning TADs to start sites of variably expressed genes...")
tss_tad_assignments = pd.Series([np.nan] * tss_loci.shape[0], 
    index=tss_loci.index.values)
for tmp_chr in tads["chr"].unique():
    print("\tWorking on {}...".format(tmp_chr))
    seg_tads = tads.loc[tads["chr"] == tmp_chr].copy()
    seg_starts = tss_loci.loc[tss_loci["chr"] == tmp_chr].copy()
    for i, tmp_tad in seg_tads.iterrows():
        tmp_hits = ~((tmp_tad["start"] > seg_starts["start"]) | \
            (tmp_tad["end"] < seg_starts["start"]))
        tmp_hits = tmp_hits[tmp_hits].index.values
        if len(tmp_hits) > 0:
            tss_tad_assignments.loc[tmp_hits] = i

tss_tad_assignments = tss_tad_assignments[\
    ~tss_tad_assignments.isna()].astype(int)

tss_loci["tad"] = ""
tss_loci.loc[tss_tad_assignments.index.values, "tad"] = \
    (tads.loc[tss_tad_assignments.values, "chr"] + ":" + \
    tads.loc[tss_tad_assignments.values, "start"].astype(str) + "-" + \
    tads.loc[tss_tad_assignments.values, "end"].astype(str)).values

tss_loci.index = tss_loci["gene"]
# tss_loci = tss_loci.loc[tss_loci["gene"].loc[tss_loci["gene"].isin(\
#     expressed_genes)].values]

Assigning TADs to start sites of variably expressed genes...
	Working on chr10...
	Working on chr11...
	Working on chr12...
	Working on chr19...
	Working on chrX...
	Working on chr8...
	Working on chr13...
	Working on chr5...
	Working on chr2...
	Working on chr6...
	Working on chr14...
	Working on chr15...
	Working on chr18...
	Working on chr16...
	Working on chr17...
	Working on chr4...
	Working on chr1...
	Working on chr3...
	Working on chr7...
	Working on chr9...
	Working on chrX_GL456233_random...
	Working on chrY...


In [36]:
## Correlate seqlet CS with gene expression within TADs
print("Finding correlations between seqlets and genes in TADs...")
corr_dict = {}
tads_without_seqlets = []
unique_tads = tss_loci["tad"].unique()
for i,tmp_tad in enumerate(unique_tads):
    print(f"{i+1}/{len(unique_tads)} tads processed", end="\r", flush=True)
    if tmp_tad != "":
        seg_gene_anno = tss_loci.loc[tss_loci["tad"] == tmp_tad]
        seg_gene_anno = seg_gene_anno.loc[\
            seg_gene_anno["gene"].isin(cite_centroids.columns.values)]
        tmp_rna = cite_centroids.loc[\
            dp_scores.columns.values,
            seg_gene_anno.index.values] 
        tmp_peaks = peaks.loc[peaks["tad"] == tmp_tad]
        tmp_peaks = (\
            tmp_peaks["chr"] + ":" + \
            tmp_peaks["start"].astype(str) + "-" + \
            tmp_peaks["end"].astype(str)).values
        tmp_seqlets = seqlets.loc[seqlets["peak"].isin(tmp_peaks)]
        if tmp_seqlets.shape[0] > 0:
            tmp_dp_scores = dp_scores.loc[tmp_seqlets.index.values]
            r_df = pearson_corr_df_to_df(tmp_rna, tmp_dp_scores.T)
            corr_res = r_df.stack().reset_index()
            corr_res.columns = ["Gene", "seqlet_idx", "r"]
            corr_dict[tmp_tad] = {}
            corr_dict[tmp_tad]["r_df"] = r_df
            corr_dict[tmp_tad]["sig_conns"] = corr_res.loc[corr_res["r"] > 0.4]
        else:
            tads_without_seqlets.append(tmp_tad)

Finding correlations between seqlets and genes in TADs...


In [39]:
## Change name of tads to prefix with 0s so it is easy to navigate
# Build temporary df to store unique tads
unique_tads = list(corr_dict.keys())
tad_re = re.compile(r'(.*):(.*)-(.*)')
unique_tads = pd.DataFrame(\
    [list(tad_re.findall(item)[0]) for item in unique_tads],
    columns=["chr", "start", "end"],
    index=unique_tads)

# Build list of zfilled ints for TAD positions
list_seg_tads = []
for tmp_chr in unique_tads["chr"].unique():
    seg_tads = unique_tads.loc[unique_tads["chr"] == tmp_chr]
    tmp_zfill = seg_tads["end"].apply(len).max()
    seg_tads.loc[:,["start", "end"]] = seg_tads.loc[:,["start", "end"]].applymap(\
        lambda x: x.zfill(tmp_zfill)).values
    list_seg_tads.append(seg_tads)
    
zfilled_tads = pd.concat(list_seg_tads)
zfilled_tads["zfill"] = zfilled_tads["chr"] + "_" + \
    zfilled_tads["start"] + "_" + zfilled_tads["end"]
tad_to_zfill = zfilled_tads[["zfill"]]
tad_to_zfill.to_csv("output/chrombpnet/modisco_merged_results/"\
    "fold_0/redo_extract_seqlets/correlate_seqlets_to_gene_expression_all_genes/"\
    "tad_to_zfill.csv", header=False, index=True)

/home/kyle/anaconda3/envs/pyInfinityFlow/lib/python3.8/site-packages/pandas/core/indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)
/home/kyle/anaconda3/envs/pyInfinityFlow/lib/python3.8/site-packages/pandas/core/indexing.py:723: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value, self.name)


In [40]:
# Pull out "significant" connections to build a separate dataframe
sig_conns = []
for tmp_tad in corr_dict:
    sig_conns.append(corr_dict[tmp_tad]["sig_conns"].copy())
    
sig_conns = pd.concat(sig_conns)

In [43]:
### Write out the correlations of TAD contained peak to gene connections
path_output = "output/chrombpnet/modisco_merged_results/fold_0/"\
    "redo_extract_seqlets/correlate_seqlets_to_gene_expression_all_genes/"
path_test_name = "seqlet_to_gene_correlation_within_tads"

# Make sure directories exist
if not os.path.isdir(path_output):
    os.mkdir(path_output)
    
if not os.path.isdir(os.path.join(path_output, path_test_name)):
    os.mkdir(os.path.join(path_output, path_test_name))
    
if not os.path.isdir(os.path.join(path_output, path_test_name, "significant_connections")):
    os.mkdir(os.path.join(path_output, path_test_name, "significant_connections"))
    
if not os.path.isdir(os.path.join(path_output, path_test_name, "r_values")):
    os.mkdir(os.path.join(path_output, path_test_name, "r_values"))
    


# Loop through TADs and write out the correlation results
for tmp_tad in unique_tads.index.values:
    corr_dict[tmp_tad]['sig_conns'].to_csv(os.path.join(\
        path_output,
        path_test_name,
        "significant_connections",
        "{}_significat_connections.csv".format(tad_to_zfill[tmp_tad])))
    corr_dict[tmp_tad]['r_df'].to_csv(os.path.join(\
        path_output,
        path_test_name,
        "r_values",
        "{}_r_values.csv".format(tad_to_zfill[tmp_tad])))

In [44]:
# Save annotation files
tss_loci.to_csv("output/chrombpnet/modisco_merged_results/fold_0/"\
    "redo_extract_seqlets/correlate_seqlets_to_gene_expression_all_genes/"\
    "gene_loci_annotation.csv",
    header=True, index=True, index_label="id")

peaks.to_csv("output/chrombpnet/modisco_merged_results/fold_0/"\
    "redo_extract_seqlets/correlate_seqlets_to_gene_expression_all_genes/"\
    "peak_annotation.csv",
    header=True, index=True, index_label="id")

In [47]:
sig_conns.to_csv("output/chrombpnet/modisco_merged_results/fold_0/"\
    "redo_extract_seqlets/correlate_seqlets_to_gene_expression_all_genes/"\
    "sig_conns.csv",
    header=True, index=False)

In [49]:
tad_to_zfill.to_csv("output/chrombpnet/modisco_merged_results/fold_0/"\
    "redo_extract_seqlets/correlate_seqlets_to_gene_expression_all_genes/"\
    "tad_to_zfill.csv",
    header=False, index=False)

In [53]:
sig_conns.shape

(30813225, 3)

In [20]:
### Read in the results from previous analysis of seqlet to gene expression cor
## Gene clusters: hierarchical clusturing across seqlet to gene r values
# Read in the gene clusters
gene_clusters = pd.read_csv(\
    "output/chrombpnet/modisco_merged_results/fold_0/redo_extract_seqlets/"\
    "seqlet_to_gene_expression_correlation_gene_clusters.csv",
    header=None, index_col=0)
gene_clusters.index.name = None
## Seqlet clusters: MarkerFinder to hierarchical gene clusters
# Read in the seqlet clusters
marker_df = pd.read_csv(\
    "output/chrombpnet/modisco_merged_results/fold_0/redo_extract_seqlets/"\
    "seqlet_to_gene_expression_correlation_seqlet_markerfinder.csv")

In [22]:
sig_conns = pd.read_csv("output/chrombpnet/modisco_merged_results/fold_0/"\
    "redo_extract_seqlets/correlate_seqlets_to_gene_expression/sig_conns.csv")

sig_conns

,Gene,seqlet_idx,r
0,Mrpl15,1856496,0.441411
1,Mrpl15,1856497,0.467160
2,Mrpl15,2458694,0.544898
3,Mrpl15,2807710,0.544757
4,Mrpl15,3763246,0.544713
...,...,...,...
22706190,Hccs,14808102,0.621244
22706191,Hccs,14808105,0.430966
22706192,Hccs,15314097,0.439063
22706193,Hccs,15796481,0.439211


In [24]:
len(sig_conns["Gene"].unique())

9034

In [27]:
"Epx" in sig_conns["Gene"].unique()

False

In [11]:
mfr, mfp = marker_finder(\
    cite_df, 
    cite_adata.obs.loc[cite_df.index.values, "lvl3"].values)

In [16]:
mfr = mfr.dropna()
stacked_mfr = mfr.stack().reset_index()
stacked_mfr.columns = ["gene", "cluster", "r"]
stacked_mfr = stacked_mfr.sort_values(by="r", ascending=False)

In [17]:
stacked_mfr

,gene,cluster,r
566521,Jchain,plasmaDC,0.951820
173401,Prg2,EoP,0.909323
1764073,Ms4a1,matB,0.882498
494819,C1qc,Mac-Nr1h3,0.880473
1300051,Epx,EoP,0.874616
...,...,...,...
408523,Rps20,immNeu-1,-0.547980
775576,Rps16,immNeu-1,-0.554078
1209532,Rpsa,immNeu-1,-0.555368
637507,Hmgb1,immNeu-1,-0.575843


In [ ]:
cite_adata.X

In [ ]:
# Read in the CITE-seq RNA counts
path_cite_data = "/media/kyle_storage/kyle_ferchen/grimes_lab_main/data/"\
    "2021_11_mouse_optimized_cite_seq/processed_files/"

adata_cite = sc.read(os.path.join(\
    path_cite_data, 
    "cite_seq_adata_rna_combined.h5ad"))
print("Computing CPTT normalized scRNA-seq from CITE-seq...")
adata_cite.X = np.log2((10000 * (adata_cite.X.T / \
    adata_cite.X.sum(axis=1).T).T) + 1)
cite_cell_anno = pd.read_csv(os.path.join(\
    path_cite_data, 
    "cite_seq_cell_annotations.csv"))
cite_cell_anno.index = cite_cell_anno["Cell_Barcode"].values

# Make pseudobulk RNA counts from cite clusters
print("Computing RNA centroids...")
shared_cells_cite = np.intersect1d(cite_cell_anno.index.values, 
    adata_cite.obs.index.values)
cite_cell_anno = cite_cell_anno.loc[shared_cells_cite]
cite_rna = {}
for tmp_cluster in cite_cell_anno["sctri_cite"].unique():
    print("\t{}...".format(tmp_cluster))
    tmp_barcodes = cite_cell_anno.loc[\
        cite_cell_anno["sctri_cite"] == tmp_cluster].index.values
    cite_rna[tmp_cluster] = pd.Series(np.asarray(\
            adata_cite[tmp_barcodes].X.mean(axis=0)).reshape(-1),
            index=adata_cite.var.index.values)

cite_rna = pd.DataFrame(cite_rna)
cite_rna = cite_rna.drop("Unknown", axis=1)
map_r7_names = pd.read_csv(os.path.join(\
    path_cite_data, 
    "map_r7-v1_to_r7-v2_names.csv"))
map_r7_names_v1_to_v2 = pd.Series(\
    map_r7_names["R7_V2"].values,
    index = map_r7_names["R7_V1"].values)

cite_rna = cite_rna.rename(map_r7_names_v1_to_v2, axis=1)
cite_rna.columns = [i.replace("-", "_") for i in cite_rna.columns.values]

In [ ]:
cite_rna.loc["Il5ra"].sort_values(ascending=False).head(20)